# Generation of $\phi(1020)$ particles

In this example, we generate min-bias events and select $\phi(1020)$ particles. We do this directly in memory, so that only the particles of interest are kept and stored to disk (as a ROOT Tree).

We use three generators and compute the pseudorapidity and transverse momentum in MeV for each particle, and whether it is prompt (has no long-lived ancestors). The generators are run in parallel.

In [1]:
from impy import models as im
from impy.util import name
from impy.kinematics import CenterOfMass
from impy.constants import MeV, GeV, TeV, long_lived
from tqdm import tqdm
import joblib
import numpy as np
import numba as nb
from particle import literals as lp, Particle
import uproot

In [2]:
@nb.vectorize
def is_long_lived(pid):
    """
    Return true if PDGID is a long-lived particle.

    Parameters
    ----------
    pid: int or array-like
        PDGID of particle.

    Returns
    -------
    bool or array of bool

    Notes
    -----
    This function can be used in numba-jitted functions and in normal Python code.
    When used from Python, it is vectorized. Inside a numba-jitted function, it
    accepts a single pid.
    """
    return abs(pid) in long_lived


@nb.njit
def is_prompt(idx, pid, imot):
    """
    Return true for prompt particles.

    Parameters
    ----------
    idx : int
        Index of current particle.
    pid : array
        Array containing the PDGID of particles.
    imot : array
        Array containing the index of the parent particles or -1
        if there is no parent.

    Returns
    -------
    bool
    """
    if len(pid) != len(imot):
        raise ValueError("pid and imot do not have same length")

    while idx != -1:
        idx = imot[idx]
        if idx == -1:
            return True
        if idx >= len(pid):
            raise IndexError("idx out of bounds")
        if is_long_lived(pid[idx]):
            return False

    return True

In [3]:
Models = [
    im.EposLHC,
    im.Sibyll23d,
    im.Pythia8
]

@joblib.delayed
def run(Model):
    kin = CenterOfMass(13 * TeV, "p", "p")
    phi_pid = lp.phi_1020.pdgid

    m = Model(kin, seed=1)

    task_id = None
    requested = 1e6
    phis = []
    n_events = 0
    with tqdm(total=requested, desc=m.pyname) as bar:
        while True:
            for event in m(100):
                n_events += 1
                n_phi = np.sum(event.pid == phi_pid)
                if n_phi > 0:
                    event = event[2:]  # cut beam particles
                    ma = event.pid == phi_pid
                    imot = event.parents[:, 0] - 1
                    prompt = np.array([is_prompt(idx, event.pid, imot) for idx in np.arange(len(event))[ma]])
                    phis.append((
                        event.eta[ma],
                        event.pt[ma] / MeV,
                        prompt
                    ))
                    bar.update(n_phi)
                    if len(phis) >= requested:
                        return phis, n_events

with joblib.Parallel(n_jobs=10, batch_size=1) as pool:
    results = pool(run(M) for M in Models)

results = {M.pyname: r for (M, r) in zip(Models, results)}

###################################################################
#        EPOS LHC      K. WERNER, T. PIEROG                       #
#                      Contact: tanguy.pierog@kit.edu             #
###################################################################
#        WARNING: This is a special retuned version !!!           #
#     Do not publish results without contacting the authors.      #
###################################################################
read from /Users/hdembinski/Extern/impy/src/impy/iamdata/epos/epos.iniev ...
read from /Users/hdembinski/Extern/impy/src/impy/iamdata/epos/epos.initl ...
 |                                                  |
 |                 S I B Y L L  2.3d                |
 |                                                  |
 |         HADRONIC INTERACTION MONTE CARLO         |
 |                        BY                        |
 |            Eun-Joo AHN, Felix RIEHN              |
 |      R. ENGEL, A. FEDYNITCH, R.S. FLETCHER,

Sibyll23d:   0%|          | 0/1000000.0 [00:00<?, ?it/s]

 SIG_AIR_INI: initializing target: (i,A)           1           0 air..
 SIG_AIR_INI: initializing target: (i,A)           2          14 nit..
 SIG_AIR_INI: initializing target: (i,A)           3          16 oxy..


Sibyll23d: 1635287it [09:08, 2981.61it/s] [09:08<10:22:36, 26.31it/s]] 
Pythia8:  21%|██        | 211402/1000000.0 [15:12<58:25, 224.99it/s]  

 gb0.le.0.  si,pt2:   234.86926269531250        222.98051660262280     


Pythia8:  28%|██▊       | 283575/1000000.0 [20:21<48:42, 245.12it/s]  

In [ ]:

with uproot.recreate("phi.root") as f:
    for k, (v, nev) in results.items():
        eta = np.concatenate([vi[0] for vi in v])
        pt = np.concatenate([vi[1] for vi in v])
        prompt = np.concatenate([vi[2] for vi in v])
        f[k] = {"eta": eta, "pt": pt, "prompt": prompt}
